### How to chat with GPT using OpenAI API

In [55]:
import os
from openai import OpenAI
from retrying import retry
from rich.console import Console

print("openai version:[%s]" % (openai.__version__))

openai version:[1.3.7]


### Locate where your key is

In [20]:
key_path = "../key/rilab_key.txt"
print("key_path:[%s]" % (key_path))

with open(key_path, "r") as f:
    OPENAI_API_KEY = f.read()
client = OpenAI(api_key=OPENAI_API_KEY)

key_path:[../key/rilab_key.txt]


### Query function

In [38]:
def query_gpt(messages: list, gpt_model="gpt-4"):
    """
    gpt_model: 'gpt-3.5-turbo' / 'gpt-4'
    """
    # Call the OpenAI API
    response = client.chat.completions.create(model=gpt_model, messages=messages)
    # Extract the response content and status code
    content = response.choices[0].message.content
    status_code = response.choices[0].finish_reason
    return content, status_code, response


print("Ready.")

Ready.


`messages`, an input to GPT, is basically a list where each item is a dictionary consists of `role` and `content`. A `role` can either be
* `system`: which defines the identity of the agent
* `user`: which states the input of a user
* `assistant`: which stores messages previously generated by the agents
More information can be found in [here](https://platform.openai.com/docs/guides/gpt/chat-completions-api).

In [39]:
role_msg = """너는 한국어로 대화에 능통한 언어모델이야"""
question = "LLM(Large Language Model)은 어떻게 작동하는지 설명해줘"
messages = [
    {"role": "system", "content": f"{role_msg}"},
    {"role": "user", "content": f"{question}"},
]
print(messages)

[{'role': 'system', 'content': '너는 한국어로 대화에 능통한 언어모델이야'}, {'role': 'user', 'content': 'LLM(Large Language Model)은 어떻게 작동하는지 설명해줘'}]


### Now let's use `GPT`

In [40]:
content, status_code, response = query_gpt(messages=messages, gpt_model="gpt-3.5-turbo")

In [44]:
print(content)

LLM(Large Language Model)은 기계 학습 기술을 사용하여 텍스트 데이터를 학습하고 생성하는 언어 모델입니다. LLM은 큰 양의 텍스트 데이터를 이용하여 문법, 문맥, 의미 등 언어의 다양한 측면을 학습하여 다양한 언어 작업을 수행할 수 있습니다.

LLM은 대량의 텍스트 데이터셋, 예를 들면 인터넷에서 수집한 언어 샘플, 문서, 뉴스 기사, 도서, 논문 등을 사용하여 학습됩니다. 이런 데이터는 언어의 다양한 관점, 문체, 주제 등을 포함하고 있어 LLM이 다양한 상황에 대응하고 자연스러운 언어 생성을 할 수 있도록 돕습니다.

LLM은 훈련 데이터의 통계적 패턴을 학습하고, 문장 구조, 단어 선택, 문맥 이해 등 다양한 언어적 특징을 파악합니다. 이를 통해 입력 문맥에 따라 다음에 올 단어나 문장을 예측하고 생성합니다. 예를 들어, "나는 오늘 밥을"까지 주어진 경우, LLM은 "먹었어요"나 "먹으러 갈래요"와 같은 다음 단어를 자연스럽게 예측할 수 있습니다.

LLM은 학습한 언어 모델의 일부를 사용자에게 제공하여 대화를 지원합니다. 사용자 입력에 따라 LLM은 문맥과 의미를 이해하고 적합한 답변을 생성합니다. LLM은 이전에 본 적 없는 문장에도 일관된 문체와 의미를 유지하며 자연스럽게 응답할 수 있습니다.

LLM의 작동 방식은 주어진 입력 문구와 해당 문구에 따른 추론과 생성 과정을 기반으로 합니다. 학습된 데이터의 통계적 패턴을 기반으로 LLM은 가능성이 높은 응답을 생성하고, 연습을 통해 보다 정확하고 유연한 응답을 구축합니다.

그러나 LLM은 단어, 문장 생성 과정에서 완벽하지는 않을 수 있습니다. 때로는 문맥을 제대로 이해하지 못하거나 부적절한 응답을 생성하기도 합니다. 따라서 LLM은 사용자의 입력에 대해 신중하게 판단하고, 생성된 응답의 적절성을 확인하는 것이 중요합니다.


In [45]:
print(status_code)

stop


In [46]:
print(response)

ChatCompletion(id='chatcmpl-8SxS4HwfoOOrL5IyBf9AFsapE4lpy', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='LLM(Large Language Model)은 기계 학습 기술을 사용하여 텍스트 데이터를 학습하고 생성하는 언어 모델입니다. LLM은 큰 양의 텍스트 데이터를 이용하여 문법, 문맥, 의미 등 언어의 다양한 측면을 학습하여 다양한 언어 작업을 수행할 수 있습니다.\n\nLLM은 대량의 텍스트 데이터셋, 예를 들면 인터넷에서 수집한 언어 샘플, 문서, 뉴스 기사, 도서, 논문 등을 사용하여 학습됩니다. 이런 데이터는 언어의 다양한 관점, 문체, 주제 등을 포함하고 있어 LLM이 다양한 상황에 대응하고 자연스러운 언어 생성을 할 수 있도록 돕습니다.\n\nLLM은 훈련 데이터의 통계적 패턴을 학습하고, 문장 구조, 단어 선택, 문맥 이해 등 다양한 언어적 특징을 파악합니다. 이를 통해 입력 문맥에 따라 다음에 올 단어나 문장을 예측하고 생성합니다. 예를 들어, "나는 오늘 밥을"까지 주어진 경우, LLM은 "먹었어요"나 "먹으러 갈래요"와 같은 다음 단어를 자연스럽게 예측할 수 있습니다.\n\nLLM은 학습한 언어 모델의 일부를 사용자에게 제공하여 대화를 지원합니다. 사용자 입력에 따라 LLM은 문맥과 의미를 이해하고 적합한 답변을 생성합니다. LLM은 이전에 본 적 없는 문장에도 일관된 문체와 의미를 유지하며 자연스럽게 응답할 수 있습니다.\n\nLLM의 작동 방식은 주어진 입력 문구와 해당 문구에 따른 추론과 생성 과정을 기반으로 합니다. 학습된 데이터의 통계적 패턴을 기반으로 LLM은 가능성이 높은 응답을 생성하고, 연습을 통해 보다 정확하고 유연한 응답을 구축합니다.\n\n그러나 LLM은 단어, 문장 생성 과정에서 완벽하지는 않을 수 있습니다. 때로는 문맥을 제대로 이해하지 못하거나 부적절한 응답을 

### Helper Class for implementing efficient chat with GPT

In [67]:
def printmd(string):
    display(Markdown(string))


class GPTchatClass:
    def __init__(
        self,
        gpt_model: str = "gpt-4",
        role_msg: str = "Your are a helpful assistant.",
        VERBOSE: str = True,
    ):
        self.gpt_model = gpt_model
        self.messages = [{"role": "system", "content": f"{role_msg}"}]
        self.init_messages = [{"role": "system", "content": f"{role_msg}"}]
        self.VERBOSE = VERBOSE
        if self.VERBOSE:
            self.console = Console()
        self.response = None

        self._setup_client()

    def _setup_client(self, key_path: str = "../"):
        key_path = "../key/rilab_key.txt"
        if self.VERBOSE:
            self.console.print(f"[bold cyan]key_path:[%s][/bold cyan]" % (key_path))

        with open(key_path, "r") as f:
            OPENAI_API_KEY = f.read()
        self.client = OpenAI(api_key=OPENAI_API_KEY)

        if self.VERBOSE:
            self.console.print(
                "[bold cyan]Chat agent using [%s] initialized with the follow role:[%s][/bold cyan]"
                % (self.gpt_model, role_msg)
            )

    def _add_message(self, role="assistant", content=""):
        """
        role: 'assistant' / 'user'
        """
        self.messages.append({"role": role, "content": content})

    def _get_response_content(self):
        if self.response:
            return self.response.choices[0].message.content
        else:
            return None

    def _get_response_status(self):
        if self.response:
            return self.response.choices[0].message.finish_reason
        else:
            return None

    def chat(
        self,
        user_msg="hi",
        PRINT_USER_MSG=True,
        PRINT_GPT_OUTPUT=True,
        RESET_CHAT=False,
        RETURN_RESPONSE=True,
    ):
        self._add_message(role="user", content=user_msg)
        self.response = self.client.chat.completions.create(
            model=self.gpt_model, messages=self.messages
        )
        # Backup response for continous chatting
        self._add_message(role="assistant", content=self._get_response_content())
        if PRINT_USER_MSG:
            self.console.print("[deep_sky_blue3][USER_MSG][/deep_sky_blue3]")
            printmd(user_msg)
        if PRINT_GPT_OUTPUT:
            self.console.print("[spring_green4][GPT_OUTPUT][/spring_green4]")
            printmd(self._get_response_content())
        # Reset
        if RESET_CHAT:
            self.messages = self.init_messages
        # Return
        if RETURN_RESPONSE:
            return self._get_response_content()


print("Ready.")

Ready.


### Now let's chat

In [68]:
GPT = GPTchatClass(gpt_model="gpt-3.5-turbo", role_msg="Your are a helpful assistant.")
PRINT_USER_MSG = True
PRINT_GPT_OUTPUT = True
RESET_CHAT = False
RETURN_RESPONSE = False
GPT.chat(
    user_msg="Who is the current president of Korea?",
    PRINT_USER_MSG=PRINT_USER_MSG,
    PRINT_GPT_OUTPUT=PRINT_GPT_OUTPUT,
    RESET_CHAT=RESET_CHAT,
    RETURN_RESPONSE=RETURN_RESPONSE,
)
GPT.chat(
    user_msg="Are you sure? I think you are outdated.",
    PRINT_USER_MSG=PRINT_USER_MSG,
    PRINT_GPT_OUTPUT=PRINT_GPT_OUTPUT,
    RESET_CHAT=RESET_CHAT,
    RETURN_RESPONSE=RETURN_RESPONSE,
)
GPT.chat(
    user_msg="Where can I get the latest information?",
    PRINT_USER_MSG=PRINT_USER_MSG,
    PRINT_GPT_OUTPUT=PRINT_GPT_OUTPUT,
    RESET_CHAT=RESET_CHAT,
    RETURN_RESPONSE=RETURN_RESPONSE,
)

key_path:[../key/rilab_key.txt]

Chat agent using  initialized with the follow role:[너는 한국어로 대화에 능통한 언어모델이야]

[USER_MSG]

Who is the current president of Korea?

[GPT_OUTPUT]

...

I apologize for any confusion caused. Please note that as an AI language model, my responses are based on information available up to September 2021. If there have been any recent changes in the political landscape, I may not be aware of them. To get the most up-to-date and accurate information, I recommend referring to reliable sources or news outlets.

[USER_MSG]

Where can I get the latest information?

[GPT_OUTPUT]

To get the latest and most accurate information, I recommend referring to reliable news sources and official government websites. Here are some reputable sources you can consider:

1. News outlets such as CNN, BBC, Reuters, Al Jazeera, and The New York Times often provide up-to-date news coverage from around the world.
2. Official government websites or press releases from institutions like the Office of the President or Ministry of Foreign Affairs of the country you are interested in can provide reliable information on political leaders and updates.
3. Local news outlets in the specific country of interest can also provide relevant and current information.

Remember to verify information from multiple sources to ensure accuracy and stay updated on any changes or developments.

### Chat with resetting everytime

In [69]:
GPT = GPTchatClass(gpt_model="gpt-3.5-turbo", role_msg="Your are a helpful assistant.")
PRINT_USER_MSG = True
PRINT_GPT_OUTPUT = True
RESET_CHAT = True
RETURN_RESPONSE = False
GPT.chat(
    user_msg="Who is the current president of Korea?",
    PRINT_USER_MSG=PRINT_USER_MSG,
    PRINT_GPT_OUTPUT=PRINT_GPT_OUTPUT,
    RESET_CHAT=RESET_CHAT,
    RETURN_RESPONSE=RETURN_RESPONSE,
)
GPT.chat(
    user_msg="Are you sure? I think you are outdated.",
    PRINT_USER_MSG=PRINT_USER_MSG,
    PRINT_GPT_OUTPUT=PRINT_GPT_OUTPUT,
    RESET_CHAT=RESET_CHAT,
    RETURN_RESPONSE=RETURN_RESPONSE,
)
GPT.chat(
    user_msg="Where can I get the latest information?",
    PRINT_USER_MSG=PRINT_USER_MSG,
    PRINT_GPT_OUTPUT=PRINT_GPT_OUTPUT,
    RESET_CHAT=RESET_CHAT,
    RETURN_RESPONSE=RETURN_RESPONSE,
)

key_path:[../key/rilab_key.txt]

Chat agent using  initialized with the follow role:[너는 한국어로 대화에 능통한 언어모델이야]

[USER_MSG]

Who is the current president of Korea?

[GPT_OUTPUT]

...

As an AI language model, I am constantly learning and adapting to provide up-to-date and relevant information. While I may not always have the most recent information on every topic, I strive to provide helpful and accurate assistance based on the data and knowledge available to me. If I am unable to assist with a specific question or topic, I will do my best to redirect you to other sources or suggest alternative approaches.

[USER_MSG]

Where can I get the latest information?

[GPT_OUTPUT]

To get the latest information on a specific topic, there are several reliable sources you can turn to:

1. News websites: Visit reputable news websites such as BBC News, CNN, Reuters, or The New York Times. These sources typically cover a wide range of current topics and provide reliable and up-to-date news articles.

2. Official government websites: For information on government policies, regulations, or updates on various topics, visit official government websites related to your country or the subject you are interested in.

3. Industry-specific websites: If you are looking for the latest information on a specific industry or field, check out websites dedicated to that industry. Many industries have specialized websites or news outlets that provide industry-specific updates and news.

4. Social media platforms: Follow verified accounts on social media platforms like Twitter or Facebook relating to the areas you want to get the latest information from. Many organizations, journalists, and experts share real-time updates and news on these platforms.

5. Online forums and communities: Engage in online forums or communities that focus on your area of interest. Often, members of these communities share and discuss the latest news and developments.

Remember, it's always important to verify information from multiple sources, especially when dealing with fast-changing or controversial topics.